# Create Winter Year Snow Cubes from IMS
- Saving seperately from annual Canada IMS cubes because of memory issues down the line on cleaning...

In [1]:
import numpy as np
#import hvplot.xarray
import xarray as xr

import sys
sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/DataAccess/Utilities')
import PreProcess_Utils as pputil

sys.path.append('C:/Users/mbonney/OneDrive - NRCan RNCan/Projects/UtilityCode/Modelling/Utilities')
import Snow_Utils as snow_util

In [2]:
from dask.distributed import LocalCluster

cluster = LocalCluster(n_workers = 10, threads_per_worker = 10, memory_limit = 0) # Refined through testing
client = cluster.get_client()
client # Can open in other window by clicking 'Dashboard' link below

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 100,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:57052,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 100
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:57100,Total threads: 10
Dashboard: http://127.0.0.1:57109/status,Memory: 0 B
Nanny: tcp://127.0.0.1:57057,


2025-02-04 15:39:05,569 - distributed.nanny - ERROR - 
Traceback (most recent call last):
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls_2025\Lib\site-packages\distributed\utils.py", line 805, in wrapper
    return await func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls_2025\Lib\site-packages\distributed\nanny.py", line 554, in _on_worker_exit
    await self._unregister()
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls_2025\Lib\site-packages\distributed\nanny.py", line 305, in _unregister
    await wait_for(
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls_2025\Lib\site-packages\distributed\utils.py", line 1910, in wait_for
    return await fut
           ^^^^^^^^^
  File "C:\Users\mbonney\Miniforge3\envs\snow_hls_2025\Lib\site-packages\distributed\core.py", line 1259, in send_recv_from_rpc
    return await send_recv(comm=comm, op=key, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\mb

# Load IMS cubes

In [3]:
year1 = 2023
year2 = 2024

path = 'D:/Canada/IMS/IMS_'

In [4]:
%%time
c1 = pputil.uploadNC(path + str(year1) + '.nc', form = 'dataarray', chunks = {'time': 1, 'band': -1, 'x': -1, 'y': -1})
#c1 = pputil.uploadNC(path + str(year1) + '.nc', chunks = {'time': -1, 'band': -1, 'x': 'auto', 'y': 'auto'}) # time = 13
c1

CPU times: total: 203 ms
Wall time: 824 ms


<xarray.DataArray (time: 365, y: 4568, x: 5372)> Size: 9GB
dask.array<open_dataset-__xarray_dataarray_variable__, shape=(365, 4568, 5372), dtype=uint8, chunksize=(1, 4568, 5372), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 43kB -2.351e+06 -2.35e+06 ... 3.019e+06 3.02e+06
  * y            (y) float64 37kB 3.843e+06 3.842e+06 ... -7.229e+05 -7.239e+05
    variable     <U18 72B ...
  * time         (time) datetime64[ns] 3kB 2023-01-01 2023-01-02 ... 2023-12-31
    spatial_ref  int64 8B ...
Attributes: (12/50)
    _FillValue:                 0
    Conventions:                CF-1.6, ACDD-1.3
    title:                      NOAA/NESDIS Interactive Multisensor Snow and ...
    summary:                    This file contains a map of snow cover and ic...
    keywords:                   EARTH SCIENCE > CRYOSPHERE > SEA ICE > ICE EX...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Ke...
    ...                         ...
    metadata_link:              https://nsidc.org/data/G02156
    naming_authority:           org.nsidc
    references:                 Ramsay, B. H. (1998). The interactive multise...
    history:                    01 January 2023 00:35Z: NetCDF file generated...
    source:                     Analysts at the United States National Ice Ce...
    comment:                    The IMS product has changed versions and reso...

In [5]:
%%time
c2 = pputil.uploadNC(path + str(year2) + '.nc', form = 'dataarray', chunks = {'time': 1, 'band': -1, 'x': -1, 'y': -1})
#c2 = pputil.uploadNC(path + str(year2) + '.nc', chunks = {'time': -1, 'band': -1, 'x': 'auto', 'y': 'auto'}) # time = 13
c2

CPU times: total: 15.6 ms
Wall time: 32.6 ms


<xarray.DataArray (time: 366, y: 4568, x: 5372)> Size: 9GB
dask.array<open_dataset-__xarray_dataarray_variable__, shape=(366, 4568, 5372), dtype=uint8, chunksize=(1, 4568, 5372), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 43kB -2.351e+06 -2.35e+06 ... 3.019e+06 3.02e+06
  * y            (y) float64 37kB 3.843e+06 3.842e+06 ... -7.229e+05 -7.239e+05
  * time         (time) datetime64[ns] 3kB 2024-01-01 2024-01-02 ... 2024-12-31
    variable     <U18 72B ...
    spatial_ref  int64 8B ...
Attributes: (12/50)
    _FillValue:                 0
    Conventions:                CF-1.6, ACDD-1.3
    title:                      NOAA/NESDIS Interactive Multisensor Snow and ...
    summary:                    This file contains a map of snow cover and ic...
    keywords:                   EARTH SCIENCE > CRYOSPHERE > SEA ICE > ICE EX...
    keywords_vocabulary:        NASA Global Change Master Directory (GCMD) Ke...
    ...                         ...
    metadata_link:              https://nsidc.org/data/G02156
    naming_authority:           org.nsidc
    references:                 Ramsay, B. H. (1998). The interactive multise...
    history:                    01 January 2024 00:35Z: NetCDF file generated...
    source:                     Analysts at the United States National Ice Ce...
    comment:                    The IMS product has changed versions and reso...

# Convert to snow cube

In [6]:
%%time
snowCube = snow_util.annualIMS2SnowCube(c1, c2, verbose = True)
snowCube

Initial IMS time-steps: Start (n = 365), End (n = 366).
Filtered to winter year (Aug 21 - Aug 20): Start (n = 133), End (n = 233).
Combined into full cube (n = 366).
Reclassifed (Snow = 1, Non-snow = 0)
CPU times: total: 203 ms
Wall time: 314 ms


<xarray.DataArray (time: 366, y: 4568, x: 5372)> Size: 9GB
dask.array<where, shape=(366, 4568, 5372), dtype=uint8, chunksize=(1, 4568, 5372), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 43kB -2.351e+06 -2.35e+06 ... 3.019e+06 3.02e+06
  * y            (y) float64 37kB 3.843e+06 3.842e+06 ... -7.229e+05 -7.239e+05
    variable     <U18 72B 'IMS_Surface_Values'
  * time         (time) datetime64[ns] 3kB 2023-08-21 2023-08-22 ... 2024-08-20
    spatial_ref  int64 8B 0

# Save as NetCDF

In [7]:
# %%time
# snowCube = pputil.loadXR(snowCube)
# snowCube # Successfully loads in here (1:05 for time = 1, 1:26 for time = -1/xy = auto, too much memory for time = 13)

In [8]:
%%time
pputil.downloadNC(snowCube, 'D:/Canada/IMS', 'IMS_snowCube_winterYear' + str(year1)[2:] + str(year2)[2:] + '.nc', form = '')

CPU times: total: 1.95 s
Wall time: 51.9 s
